Forward Kinematics

In [51]:
import numpy as np
import sympy as sym
from sympy import symbols, cos, sin, pi, Matrix

#symbolic variables
theta1, theta2, theta3, theta4, theta5, theta6 = symbols('theta1 theta2 theta3 theta4 theta5 theta6')
L1, L2, L3, L4, L5 = symbols('L1 L2 L3 L4 L5')

#rotation Matrices
def Rx(phi):
    return Matrix([[1, 0, 0, 0],
                   [0, cos(phi), -sin(phi), 0],
                   [0, sin(phi), cos(phi), 0],
                   [0, 0, 0, 1]])

def Rz(phi):
    return Matrix([[cos(phi), -sin(phi), 0, 0],
                   [sin(phi), cos(phi), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

def DH2A(d, theta, a, alpha):
    """DH transformation matrix """
    Tb = Matrix([[1, 0, 0, 0],
                 [0, 1, 0, 0],
                 [0, 0, 1, d],
                 [0, 0, 0, 1]])
    
    Ttht = Rz(theta)
    Ta = Matrix([[1, 0, 0, a],
                 [0, 1, 0, 0],
                 [0, 0, 1, 0],
                 [0, 0, 0, 1]])
    
    Talp = Rx(alpha)
    
    return Tb * Ttht * Ta * Talp



Transformation matrices

In [52]:
A1 = DH2A(L1, theta1, 0, pi/2)
A1[0,1], A1[1,1], A1[2,2] = 0, 0, 0
print(A1)


Matrix([[cos(theta1), 0, sin(theta1), 0], [sin(theta1), 0, -cos(theta1), 0], [0, 1, 0, L1], [0, 0, 0, 1]])


In [53]:
A2 = DH2A(0, theta2 + pi/2, L2, 0)
print(A2)

Matrix([[-sin(theta2), -cos(theta2), 0, -L2*sin(theta2)], [cos(theta2), -sin(theta2), 0, L2*cos(theta2)], [0, 0, 1, 0], [0, 0, 0, 1]])


In [54]:
A3 = DH2A(0, theta3, L3, 0)

In [55]:
A4a = DH2A(0, theta4 + pi/2, 0, pi/2)
A4b = DH2A(L4, 0, 0, 0)
A4a[0,1], A4a[1,1], A4a[2,2] = 0, 0, 0
A4 = A4a * A4b

In [56]:
A5 = DH2A(L5, theta5, 0, 0)

In [57]:
#final transformation matrices
T30 = A1 @ A2 @ A3
T53 = A4 @ A5
T50 = T30 @ T53
# print(T50)

Validation with numerical values

In [58]:
consts = {L1: 25.929, L2: 81.379, L3: 83.009, L4: 77, L5: 106}
vals = {theta1: 0, theta2: 0, theta3: 0, theta4: -90*pi/180, theta5: 90*pi/180}

#substituting values and evaluating the final transformation matrix
T_evaluated = T50.subs({**consts, **vals}).evalf(4)
print(T_evaluated)

Matrix([[0, 0, 1.000, 183.0], [-1.000, 0, 0, 0], [0, -1.000, 0, 190.3], [0, 0, 0, 1.000]])


Finding Position and Orientation using Euler angles

In [59]:
T = sym.Matrix(T_evaluated)
R = T_evaluated[:3, :3]

pitch = sym.atan2(-R[2, 0], sym.sqrt(R[0, 0]**2 + R[1, 0]**2))
yaw = sym.Piecewise(
    (sym.atan2(R[1, 0], R[0, 0]), abs(pitch) < sym.pi / 2),
    (sym.atan2(-R[0, 1], R[1, 1]), True)
)
roll = sym.Piecewise(
    (sym.atan2(R[2, 1], R[2, 2]), abs(pitch) < sym.pi / 2),
    (0, True)
)
position = T_evaluated[:3, 3]

print("orientation")
print("yaw", yaw.evalf())
print("pitch", pitch.evalf())
print("roll", roll.evalf())

print("\nposition vector:")
print(position.evalf())

orientation
yaw -1.57079632679490
pitch 0
roll -1.57079632679490

position vector:
Matrix([[183.000000000000], [0], [190.316406250000]])
